In [ ]:
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt

df1 = pd.read_csv('data.csv')

df1.head(5)

In [ ]:
d = []

principles = ['F1', 'F2', 'F3', 'F4', 'A1.1', 'A1.2', 'A2', 'I1', 'I2', 'I3', 'R1.1', 'R1.2']
years = [2019, 2020, 2021]

for year in years:
    d1 = {}
    d.append(d1)
    resources = Series(dtype='float64')
    for principle in principles:
        selection = df1[(df1['p'] == principle) & (df1['y'] == year)]['res']
        resources = pd.concat([resources, selection])
        unique = len(set(selection)) # Number of distinct resources in current use in communities for the given year and principle
        total = len(selection) # Number of resources in current use communities for the given year and principle
        fraction = unique/total if total > 0 else 0
        # 5/5 means that all communities use different resources
        # 1/10 means that all communities use the same resource
        # The lower the fraction the better
        #print('{} {} {} {} {}'.format(year, principle, unique, total, fraction))
        d1[principle] = fraction
    d1['R1.3'] = len(set(resources))/len(resources)
        

df2 = pd.DataFrame(d)
df2.index = years

In [ ]:
df3 = df2.T
df3.head(3)

In [ ]:
diff = []

# A negative difference means that there has been convergence along the principle
for index, row in df3.iterrows():
    diff.append(row[2021]-row[2019])

df3['Difference'] = diff

df3.sort_values(by='Difference', ascending=True)

In [ ]:
# Good examples: F4, I1
principle = 'I1'

y1 = df1[(df1['p'] == principle) & (df1['y'] == 2019)]['res'].value_counts()
y2 = df1[(df1['p'] == principle) & (df1['y'] == 2021)]['res'].value_counts()

df4 = pd.DataFrame({'2019': y1, '2021': y2}, dtype='int64')

#df4 = df4.dropna()

# Drop those that have same counts in 2019 and 2021
df4 = df4[df4['2019'] != df4['2021']]

df4 = df4.sort_values(by='2019', ascending=False)

df4.head(5)

In [ ]:
ax = df4.plot.bar(figsize=(12,8))
plt.legend(fontsize=20)
plt.xlabel('Resources', fontsize=20)
plt.ylabel('Count', fontsize=20)
plt.xticks(fontsize=16, rotation=45)
plt.yticks(fontsize=16)
plt.show()